# 🐍 mssql-python: Native Python Objects with SQL Server

**Microsoft's new Python Driver for Azure SQL/SQL Server** 

---

## 📦 Installation

```bash
pip install mssql-python
```

That's it! No ODBC drivers, no additional dependencies. Just pure Python.

---

In [14]:
# 📦 Import the library
import mssql_python
import os
from dotenv import load_dotenv
from datetime import datetime, date
from decimal import Decimal

load_dotenv()

print(f"✅ mssql-python version: {mssql_python.__version__}")

✅ mssql-python version: 1.0.0


In [15]:
# 🔧 Connection string - using environment variable for password
SQL_PASSWORD = os.getenv("SQL_PASSWORD")

CONN_STRING = (
    "Server=localhost,1433;"
    "Database=AdventureWorks2022;"
    "UID=sa;"
    f"Pwd={SQL_PASSWORD};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

print("✅ Connection string configured")

✅ Connection string configured


## 🏗️ Create Demo Table

Let's create a table with various data types to showcase native Python object handling.

In [16]:
# 🏗️ Create the azure_ug_demo table
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

# Drop if exists, then create fresh
cursor.execute("""
    IF OBJECT_ID('azure_ug_demo', 'U') IS NOT NULL
        DROP TABLE azure_ug_demo
""")

cursor.execute("""
    CREATE TABLE azure_ug_demo (
        id INT IDENTITY(1,1) PRIMARY KEY,
        name NVARCHAR(100),
        age INT,
        salary DECIMAL(10, 2),
        is_active BIT,
        hire_date DATE,
        created_at DATETIME
    )
""")

conn.commit()
print("✅ Table 'azure_ug_demo' created!")
conn.close()

✅ Table 'azure_ug_demo' created!


## ➕ Insert Data Using Native Python Objects

mssql-python automatically handles type conversion from Python to SQL Server:

| Python Type | SQL Server Type |
|-------------|----------------|
| `str` | NVARCHAR |
| `int` | INT |
| `Decimal` | DECIMAL |
| `bool` | BIT |
| `date` | DATE |
| `datetime` | DATETIME |

In [17]:
# ➕ Insert single row with native Python objects
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

# Native Python objects - no manual conversion needed!
cursor.execute(
    """
    INSERT INTO azure_ug_demo (name, age, salary, is_active, hire_date, created_at)
    VALUES (?, ?, ?, ?, ?, ?)
    """,
    "Alice Smith",           # str → NVARCHAR
    30,                       # int → INT
    Decimal("75000.50"),     # Decimal → DECIMAL
    True,                     # bool → BIT
    date(2023, 6, 15),       # date → DATE
    datetime.now()           # datetime → DATETIME
)

conn.commit()
print("✅ Single row inserted!")
conn.close()

✅ Single row inserted!


In [18]:
# ➕ Insert multiple rows using executemany
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

# List of tuples with native Python objects
employees = [
    ("Bob Johnson", 45, Decimal("95000.00"), True, date(2020, 1, 10), datetime.now()),
    ("Carol Williams", 28, Decimal("62000.75"), True, date(2024, 3, 1), datetime.now()),
    ("David Brown", 35, Decimal("82500.25"), False, date(2019, 8, 22), datetime.now()),
    ("Eve Davis", 52, Decimal("110000.00"), True, date(2015, 11, 5), datetime.now()),
]

cursor.executemany(
    """
    INSERT INTO azure_ug_demo (name, age, salary, is_active, hire_date, created_at)
    VALUES (?, ?, ?, ?, ?, ?)
    """,
    employees
)

conn.commit()
print(f"✅ {len(employees)} rows inserted!")
conn.close()

✅ 4 rows inserted!


## 📖 Fetch Data as Native Python Objects

When you fetch data, mssql-python automatically converts SQL Server types back to Python objects!

In [19]:
# 📖 Fetch all rows
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

cursor.execute("SELECT * FROM azure_ug_demo")
rows = cursor.fetchall()

print(f"📊 Retrieved {len(rows)} rows:\n")
print("-" * 80)

for row in rows:
    print(f"ID: {row[0]}")
    print(f"  Name: {row[1]} (type: {type(row[1]).__name__})")
    print(f"  Age: {row[2]} (type: {type(row[2]).__name__})")
    print(f"  Salary: ${row[3]:,.2f} (type: {type(row[3]).__name__})")
    print(f"  Active: {row[4]} (type: {type(row[4]).__name__})")
    print(f"  Hire Date: {row[5]} (type: {type(row[5]).__name__})")
    print(f"  Created: {row[6]} (type: {type(row[6]).__name__})")
    print("-" * 80)

conn.close()

📊 Retrieved 5 rows:

--------------------------------------------------------------------------------
ID: 1
  Name: Alice Smith (type: str)
  Age: 30 (type: int)
  Salary: $75,000.50 (type: Decimal)
  Active: True (type: bool)
  Hire Date: 2023-06-15 (type: date)
  Created: 2025-12-13 09:57:25.323000 (type: datetime)
--------------------------------------------------------------------------------
ID: 2
  Name: Bob Johnson (type: str)
  Age: 45 (type: int)
  Salary: $95,000.00 (type: Decimal)
  Active: True (type: bool)
  Hire Date: 2020-01-10 (type: date)
  Created: 2025-12-13 09:57:31.693000 (type: datetime)
--------------------------------------------------------------------------------
ID: 3
  Name: Carol Williams (type: str)
  Age: 28 (type: int)
  Salary: $62,000.75 (type: Decimal)
  Active: True (type: bool)
  Hire Date: 2024-03-01 (type: date)
  Created: 2025-12-13 09:57:31.693000 (type: datetime)
--------------------------------------------------------------------------------
I

In [20]:
# 📖 Fetch with parameterized query
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

# Query using native Python types as parameters
min_salary = Decimal("80000.00")
active_only = True

cursor.execute(
    """
    SELECT name, salary, hire_date 
    FROM azure_ug_demo 
    WHERE salary >= ? AND is_active = ?
    ORDER BY salary DESC
    """,
    min_salary,
    active_only
)

print(f"💰 Employees earning ≥ ${min_salary:,.2f} (active only):\n")
for row in cursor.fetchall():
    print(f"  {row[0]}: ${row[1]:,.2f} (hired: {row[2]})")

conn.close()

💰 Employees earning ≥ $80,000.00 (active only):

  Eve Davis: $110,000.00 (hired: 2015-11-05)
  Bob Johnson: $95,000.00 (hired: 2020-01-10)


## 🧹 Cleanup

In [21]:
# 🧹 Drop the demo table
conn = mssql_python.connect(CONN_STRING)
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS azure_ug_demo")
conn.commit()

print("✅ Table 'azure_ug_demo' dropped!")
conn.close()

✅ Table 'azure_ug_demo' dropped!


---

## 🎯 Key Takeaways

1. **Zero Config** - Just `pip install mssql-python`, no ODBC drivers needed
2. **Native Types** - Python objects map directly to SQL Server types
3. **Familiar API** - DB-API 2.0 compliant (same patterns as pyodbc, sqlite3)
4. **Parameterized Queries** - Use `?` placeholders with native Python values

**Learn more:** [github.com/microsoft/mssql-python](https://github.com/microsoft/mssql-python)